In [ ]:
# TODO: Notebook mit shapes und ditanzen erstellen

### 1. Load the Fish Data, split into train and test set, and standardize data.

In [1]:
import pandas as pd
fish_data = pd.read_csv('fish2.csv')
fish_data

,Species,Length1,Length2,Length3,Height,Width,Weight
0,Bream,23.2,25.4,30.0,11.5200,4.0200,242.0
1,Bream,24.0,26.3,31.2,12.4800,4.3056,290.0
2,Bream,23.9,26.5,31.1,12.3778,4.6961,340.0
3,Bream,26.3,29.0,33.5,12.7300,4.4555,363.0
4,Bream,26.5,29.0,34.0,12.4440,5.1340,430.0
...,...,...,...,...,...,...,...
154,Smelt,11.5,12.2,13.4,2.0904,1.3936,12.2
155,Smelt,11.7,12.4,13.5,2.4300,1.2690,13.4
156,Smelt,12.1,13.0,13.8,2.2770,1.2558,12.2
157,Smelt,13.2,14.3,15.2,2.8728,2.0672,19.7


In [2]:
## Create Dummy Variables and convert to torch
import torch
X = fish_data[['Length1', 'Length2', 'Length3', 'Height', 'Width', 'Species']]
X = torch.tensor(pd.get_dummies(X, dtype=float).to_numpy(), dtype=torch.float)
y = torch.tensor(fish_data[['Weight']].to_numpy(), dtype=torch.float)

In [3]:
## Split into train and test set
generator = torch.Generator().manual_seed(42)
idx = torch.randperm(X.shape[0], generator=generator)
X_train, X_test = X[idx[:127],:], X[idx[127:],:]
y_train, y_test = y[idx[:127],:], y[idx[127:],:]

In [4]:
## Standardize Data
# Lambda Function
standardize = lambda z, z_mean, z_std: (z - z_mean) / (z_std + 1e-5) 

# Standardize X
x_mean, x_std = X_train[:,:5].mean(0, keepdims=True), X_train[:,:5].std(0, keepdims=True)
X_train[:,:5], X_test[:,:5] = standardize(X_train[:,:5], x_mean, x_std), standardize(X_test[:,:5], x_mean, x_std)
# Standardize y
y_mean, y_std = y_train.mean(0, keepdims=True), y_train.std(0, keepdims=True)
y_train = standardize(y_train, y_mean, y_std)

### 2. Create k Nearest Neighbor Regressor class

In [5]:
class kNNRegressor:
    # k: Number of k Nearest Neighbors
    # p: L_p distance function
    # Note: output label is simple average of k Nearest Neighbors
    def __init__(self, k, p=2.0):
        self.k = k
        self.p = p
        self.train_x = None
        self.train_y = None
    
    def fit(self, X, y):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        """  
        self.train_x = X
        self.train_y = y
        return self
        
    def predict(self, X, mean=None, std=None):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        """
        # Calculate distance matrix
        dist = (self.train_x.unsqueeze(0) - X.unsqueeze(1)).abs()
        dist = dist.pow(self.p).mean(2).pow(1./self.p)
        
        # Get k nearest neighbots
        _, idx_arg = torch.topk(dist, k=self.k, dim=1, largest=False)
        
        # Collect labels
        train_y = self.train_y.repeat(1, self.k) # First repeat train labels to match idx_arg
        predictions = train_y.gather(0, idx_arg) # Next, gather train labels
        predictions = predictions.mean(1, keepdims=True) # Finally, calculate mean
        
        # Recompute unstandardized data
        if mean is not None and std is not None:
            predictions = predictions * std + mean
        return predictions

In [6]:
k, p = 5, 2.
knn_model = kNNRegressor(k, p=p).fit(X_train, y_train)
y_pred = knn_model.predict(X_test, y_mean, y_std)
mse = (y_pred - y_test).pow(2.).mean()
print(f'The MSE of the {k}-NN model with L_{p}-Distance is: {mse:10.4f}')

The MSE of the 5-NN model with L_2.0-Distance is:  4855.2710


### 3. Create Linear Regressor class

In [7]:
class LinearRegression:
    def __init__(self):
        self.weight = None
            
    def fit(self, X, y):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        """        
        
        # prepend a column of ones
        ones = torch.zeros(X.shape[0], 1)
        X = torch.cat((ones, X), axis=1)           

        # compute weights
        XtX_pinv = torch.linalg.pinv(torch.mm(X.T, X))
        Xty = torch.mm(X.T,y)
        
        # dot: matrix multiplication
        self.weight = torch.mm(XtX_pinv, Xty)
        return self
                
    def predict(self, X, mean=None, std=None):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        """                     
        # prepend a column of ones
        ones = torch.zeros(X.shape[0], 1)
        X = torch.cat((ones, X), axis=1)

        # compute predictions
        predictions = torch.mm(X, self.weight)
        
        # Recompute unstandardized data
        if mean is not None and std is not None:
            predictions = predictions * std + mean
        return predictions

In [8]:
lr_model = LinearRegression().fit(X_train, y_train)
y_pred = lr_model.predict(X_test, y_mean, y_std)
mse = (y_pred - y_test).pow(2.).mean()
print(f'The MSE of the linear regression model is: {mse:10.4f}')

The MSE of the linear regression model is:  6102.0005


### 4. Create a Linear Regressor class, but this time use PyTorch autograd function to calculate the gradient.

In [9]:
import torch.nn as nn

class LinearRegressionGradientDescent:
    def __init__(self, input_features, lr=0.1, epochs=1000):
        self.weight = nn.Parameter(torch.zeros(input_features + 1, 1))
        self.lr, self.epochs = lr, epochs
        
    def fit(self, X, y):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        y : Array of shape [n_samples, 1]
        """        
        for _ in range(self.epochs):
            y_pred = self.forward(X)
            loss = (y - y_pred).abs().mean()
            loss.backward()
            with torch.no_grad():
                self.weight -= self.lr * self.weight.grad
            self.weight.grad = None
        return self
    
    def forward(self, X):
        """
        Parameters
        ----------
        X : Array of shape [n_samples, n_features]
        """                     
        # prepend a column of ones
        ones = torch.zeros(X.shape[0], 1)
        X = torch.cat((ones, X), axis=1)

        # compute predictions
        predictions = torch.mm(X, self.weight)

        return predictions
    
    def predict(self, X, mean=None, std=None):
        # Execute forward pass
        predictions = self.forward(X)
        
        # Recompute unstandardized data
        if mean is not None and std is not None:
            predictions = predictions * std + mean
            
        return predictions

In [10]:
lrd_model = LinearRegressionGradientDescent(12).fit(X_train, y_train)
y_pred = lrd_model.predict(X_test, y_mean, y_std)
mse = (y_pred - y_test).pow(2.).mean()
print(f'The MSE of the gradient descent linear regression model is: {mse:10.4f}')

The MSE of the gradient descent linear regression model is:  4764.8892
